In [12]:
from pandas import read_csv, DataFrame
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd
import numpy as np

In [13]:
train = read_csv('train.csv')
test_x = read_csv('test_x.csv')
train['average_bill'] = train.average_bill.fillna(value=500)
test_x['average_bill'] = test_x.average_bill.fillna(value=500)

In [14]:
def get_x(df):
    x = df
    one_hot_rubrics = pd.get_dummies(x['rubrics'])
    one_hot_org_city = pd.get_dummies(x['org_city'])
    one_hot_user_city = pd.get_dummies(x['user_city'])
    one_hot_user_city['msku'] = one_hot_user_city['msk']
    one_hot_user_city['spbu'] = one_hot_user_city['spb']
    one_hot_user_city = one_hot_user_city.drop(['msk', 'spb'], axis = 1)
    x = x.drop(['rubrics', 'user_city', 'org_city'], axis=1).join(one_hot_rubrics).join(one_hot_org_city).join(one_hot_user_city)
    x['advantageous_delivery'] = 1 * ((x['free_delivery'] == 1) & (x['minimum_order'] == 0))
    x['convenient_for_work'] = 1 * ((x['air_conditioning'] == 1) & (x['wi_fi'] == 1))
    x['around_the_clock_food'] = 1 * ((x['around_the_clock_work1'] == 1) & (x['food_delivery'] == 1))
    x['user_id'] = x['user_id'].max() - x['user_id']
    x['org_id'] = x['org_id'].max() - x['org_id'] # similar org_id
    x['average_bill_500_1500'] =  1 * ((x['average_bill'] >= 500)  & x.average_bill <= 1500)
    x['average_bill_1500_3000'] = 1 * ((x.average_bill > 1500) & (x.average_bill <= 3000)) 
    x['average_bill_3000_'] = 1 * (x.average_bill > 3000)
    x['rating'] = x.rating.max() - x.rating
    x['average_bill'] = (x.average_bill.max() - x.average_bill) / 500
    x['user_id'] = x.user_id * 1e-18
    x['org_id'] = x.org_id * 1e-18
    return x

In [15]:
train  = train[train.average_bill <= 10000]
x_train = get_x(train).drop(['rating_org'], axis=1)
y_train = get_x(train)[['rating_org']]
x_train

,user_id,org_id,rating,ts,average_bill,food_delivery,breakfast,takeaway,summer_terrace,wi_fi,...,msk,spb,msku,spbu,advantageous_delivery,convenient_for_work,around_the_clock_food,average_bill_500_1500,average_bill_1500_3000,average_bill_3000_
0,6.263866,6.498911,1.0,6.77,17.0,1,1,1,0,1,...,1,0,1,0,0,0,0,1,0,0
1,15.650111,12.899453,0.0,5.77,18.0,1,0,1,1,1,...,1,0,1,0,0,0,0,1,0,0
2,7.899831,14.993828,1.0,7.16,19.0,1,1,1,1,1,...,1,0,1,0,0,0,0,1,0,0
3,8.478913,8.955272,2.0,6.39,18.0,1,1,1,1,1,...,0,1,0,1,0,0,0,1,0,0
4,7.399836,13.211058,0.0,6.69,19.0,1,1,1,1,1,...,1,0,1,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73990,5.919712,15.497180,3.0,5.17,19.0,1,1,0,1,1,...,1,0,1,0,0,0,0,1,0,0
73991,0.654272,5.220085,0.0,4.55,19.0,0,0,0,0,1,...,1,0,1,0,0,0,0,1,0,0
73992,10.994330,17.310038,0.0,10.32,18.0,1,1,1,1,1,...,1,0,1,0,0,0,0,1,0,0
73993,18.078334,15.946344,0.0,0.27,19.0,0,0,0,0,0,...,1,0,1,0,0,0,0,1,0,0


In [16]:
model = KNeighborsRegressor(n_neighbors=int(np.sqrt(len(x_train))), weights="distance")
model.fit(x_train, y_train)
x_test = get_x(test_x)
y = model.predict(x_test)

In [17]:
ans = pd.DataFrame()
ans['id'] = list(range(len(y.squeeze())))
ans['rating_org'] = np.array(y.squeeze())
ans = ans.set_index('id')
ans.to_csv('my_submission.csv')
ans

,rating_org
id,
0,4.278006
1,4.219149
2,4.318563
3,4.299518
4,4.353025
...,...
24660,4.265259
24661,4.453894
24662,4.130944
